In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import nltk

# Veri setini yükle
df = pd.read_csv('onislenmis_veriler.csv')
S
# 'cumleler' sütunundaki değerleri string'e dönüştür
df['cumleler'] = df['cumleler'].astype(str)

# DataFrame'den metin verilerini çıkarıp tokenleme yapın
tokenized_sentences = [word_tokenize(sentence) for sentence in df['cumleler']]

# Word2Vec modelini eğit
vector_size = 100
window = 5
min_count = 1
workers = 4
model = Word2Vec(tokenized_sentences, vector_size=vector_size, window=window, min_count=min_count, workers=workers)

# En sık geçen kelimeleri listeleme
top_words_count = 20
cok_gecen_kelimeler = model.wv.index_to_key[:top_words_count]

# En sık geçen kelimelerin frekansını bul
for word in cok_gecen_kelimeler:
    count = model.wv.get_vecattr(word, "count")
    print(f"'{word}' kelimesi {count} kez geçiyor.")

'kisi' kelimesi 449 kez geçiyor.
'yok' kelimesi 445 kez geçiyor.
'adam' kelimesi 438 kez geçiyor.
'buyuk' kelimesi 401 kez geçiyor.
'erdogan' kelimesi 388 kez geçiyor.
'cumhurbaskani' kelimesi 370 kez geçiyor.
'tayyip' kelimesi 342 kez geçiyor.
'turkiye' kelimesi 340 kez geçiyor.
'son' kelimesi 335 kez geçiyor.
'tek' kelimesi 307 kez geçiyor.
'basbakan' kelimesi 299 kez geçiyor.
'oy' kelimesi 295 kez geçiyor.
'iyi' kelimesi 289 kez geçiyor.
'gun' kelimesi 287 kez geçiyor.
'zaman' kelimesi 259 kez geçiyor.
'ilk' kelimesi 240 kez geçiyor.
'bugun' kelimesi 240 kez geçiyor.
'recep' kelimesi 227 kez geçiyor.
'ulkenin' kelimesi 226 kez geçiyor.
'secim' kelimesi 218 kez geçiyor.


In [28]:
# En sık geçen kelimelerin her biri için en benzer 5 kelimeyi bul
for word in cok_gecen_kelimeler:
    similar_words = model.wv.most_similar(word, topn=5)
    print(f"\n'{word}' kelimesinin en benzer 5 kelimesi: {similar_words}")


'kisi' kelimesinin en benzer 5 kelimesi: [('bugun', 0.9989054799079895), ('son', 0.9988786578178406), ('iyi', 0.9988134503364563), ('zaman', 0.9988111257553101), ('oy', 0.9987993240356445)]

'yok' kelimesinin en benzer 5 kelimesi: [('gun', 0.9989348649978638), ('oy', 0.9987810254096985), ('kisi', 0.9987764954566956), ('adam', 0.998734712600708), ('zaman', 0.9986973404884338)]

'adam' kelimesinin en benzer 5 kelimesi: [('son', 0.9988461136817932), ('oy', 0.9987444281578064), ('yok', 0.9987347722053528), ('kisi', 0.9986304044723511), ('ulkenin', 0.9985170364379883)]

'buyuk' kelimesinin en benzer 5 kelimesi: [('yok', 0.9986286163330078), ('zaman', 0.9986258745193481), ('kisi', 0.9986160397529602), ('son', 0.9985594749450684), ('oy', 0.9985513687133789)]

'erdogan' kelimesinin en benzer 5 kelimesi: [('erdoganin', 0.9737420678138733), ('erdogani', 0.9694381356239319), ('tayyip', 0.9691774249076843), ('erdogana', 0.9632028341293335), ('adi', 0.9469299912452698)]

'cumhurbaskani' kelimesini

In [23]:
secilen_cumleler = [
    "sevenin sevdigi sevmeyenin sevmedigi lider erdogana karsi notr hayatimda gormedim",
    "hayatimin genclik doneminin kendisinin iktidarda zamana denk geldigi siyasetci",
    "yasadigimiz gunun sorumlusu kendisidir",
    "turk siyasal tarihinin sandik anlaminda basarili ismi kanitlayan cambaz siyasetci",
    "cumhurbaskani dedigimiz kisi ilk tarafsiz olmalidir meydanlara inip miting resmen partisi oy topladi"
]

In [31]:
df['cumleler']=df['cumleler'].apply(lambda x:x if len(x.split()) >3 else '')

# 'cumleler' sütununu seçme
cumleler = df['cumleler'].tolist()

# TfidfVectorizer kullanarak metinleri vektörlere dönüştürme
vectorizer = TfidfVectorizer()
vektorler = vectorizer.fit_transform(cumleler + secilen_cumleler)

# Cümleler arasındaki benzerliği ölçme
benzerlik_matrisi = cosine_similarity(vektorler)

for i, ornek_cümle in enumerate(secilen_cumleler):
    benzerlik_skorları = benzerlik_matrisi[len(cumleler) + i][:len(cumleler)]
    
    # Kendisiyle eşleşen benzerlik skorlarını filtreleme
    benzerlik_skorları = [(index, skor) for index, skor in enumerate(benzerlik_skorları) if cumleler[index] != ornek_cümle]
    
    # Benzerlik skorlarını sıralayıp en yüksek 3 benzerlik skorunu alıyor.
    en_benzer_cümleler_index = sorted(benzerlik_skorları, key=lambda x: x[1], reverse=True)[:3]
    
    print(f"\nDerlemdeki cümle: {ornek_cümle}")
    print("En benzer üç cümle:")
    for index, skor in en_benzer_cümleler_index:
        print(f"- {cumleler[index]} ")


Derlemdeki cümle: sevenin sevdigi sevmeyenin sevmedigi lider erdogana karsi notr hayatimda gormedim
En benzer üç cümle:
- ortada ugrasmasina karsi notr kisilerin ogrendigim sahis 
- hayatimda dogru konusan insan evladi gormedim yuzun kizarmiyor kardesim 
- kusaginin sevmedigi insan olabilir 

Derlemdeki cümle: hayatimin genclik doneminin kendisinin iktidarda zamana denk geldigi siyasetci
En benzer üç cümle:
- merak gece hayatimin guzel 
- muhalefet sayesinde olene iktidarda kalmaya devam siyasetci 
- goreve geldigi yilin ulke tarihinin buyuk krizi denk gelen noktasi yili baz alan gercegini kenara 

Derlemdeki cümle: yasadigimiz gunun sorumlusu kendisidir
En benzer üç cümle:
- ekiplerin hizli bicimde oraya ulasip yardima sorumlusu kendisidir 
- bugun yasadigimiz kriz buhranin 
- son gundur yasadigimiz masada iddia edebilirim 

Derlemdeki cümle: turk siyasal tarihinin sandik anlaminda basarili ismi kanitlayan cambaz siyasetci
En benzer üç cümle:
- cumhuriyet tarihinin basarili siyasetci

In [25]:
metinler = df['cumleler'].tolist()

# Duygu analizi yapma ve sonuçları saklama
duygu_analizleri = []

for metin in metinler:
    blob = TextBlob(metin)
    duygu_durumu = blob.sentiment.polarity
    
    if duygu_durumu > 0:
        sonuc = "Olumlu"
    elif duygu_durumu < 0:
        sonuc = "Olumsuz"
    else:
        sonuc = "Nötr"
    
    duygu_analizi = {"Metin": metin, "Duygu Durumu": duygu_durumu, "Sonuç": sonuc}
    duygu_analizleri.append(duygu_analizi)

# Duygu analizi sonuçlarını DataFrame'e çevirme
duygu_analiz_df = pd.DataFrame(duygu_analizleri)

# Sonuçları yazdırma
print(duygu_analiz_df)

                                                   Metin  Duygu Durumu Sonuç
0      bilmem belli donem secilmis ust duzey devlet m...           0.0  Nötr
1      cumhurbaskanligi yeminin anlamini bilmeyen gec...           0.0  Nötr
2                                                                  0.0  Nötr
3                        baskanlik sistemini ulkeye kisi           0.0  Nötr
4               dedem tvde gordugunde cikti dolu demiyor           0.0  Nötr
...                                                  ...           ...   ...
13937                 akp iktidarinda yabancilara toprak           0.0  Nötr
13938           istanbul topraklarini yabancilara satmak           0.0  Nötr
13939                                                              0.0  Nötr
13940           yabancilarin bol bol toprak satin almasi           0.0  Nötr
13941                                                              0.0  Nötr

[13942 rows x 3 columns]
